##### multi class logistic regression isn't good for getting weights

In [ ]:
import pandas as pd

nearest_d = pd.read_csv("~/nfl-data-bowl/Sample_Data/Raw/nearest_defender_at_snap.csv")
thru_snap = pd.read_csv("~/nfl-data-bowl/Sample_Data/Raw/playersThroughSnap.csv")
off_mot = pd.read_csv("~/nfl-data-bowl/Sample_Data/Raw/processed_offensive_motion_plays.csv")
v_late  = pd.read_csv("~/nfl-data-bowl/Sample_Data/Raw/velocity_late_shifts.csv")
play_labels = pd.read_csv("~/nfl-data-bowl/Sample_Data/Raw/playlabels.csv")



thru_snap.rename(columns={'Game ID': 'gameId'}, inplace=True)
thru_snap.rename(columns={'Play ID': 'playId'}, inplace=True)

off_mot.rename(columns={'gameID': 'gameId'}, inplace=True)
off_mot.rename(columns={'playID': 'playId'}, inplace=True)


#### fuads file

In [11]:
import pandas as pd

aggregated_rows = []

for (gameId, playId), group in v_late.groupby(['gameId', 'playId']):
    team_total_distance = group['teamTotalDistance'].iloc[0]
    team_average_speed = group['averageSpeed'].sum()
    num_players_moved = group['numPlayerMoved'].iloc[0]

    aggregated_rows.append({
        'gameId': gameId,
        'playId': playId,
        'teamTotalDistance': team_total_distance,
        'teamAverageSpeed': team_average_speed,
        'numPlayerMoved': num_players_moved
    })

v_late_mod = pd.DataFrame(aggregated_rows)

In [14]:
print(nearest_d.shape)
print(thru_snap.shape)
print(off_mot.shape)
print(v_late_mod.shape)
print(play_labels.shape)

(16124, 8)
(3850, 6)
(16124, 10)
(12814, 5)
(15740, 3)


In [40]:
merged_df = nearest_d.merge(thru_snap, on=['gameId', 'playId'], how='left')
merged_df = merged_df.merge(off_mot, on=['gameId', 'playId'], how='inner')
merged_df = merged_df.merge(v_late_mod, on=['gameId', 'playId'], how='inner')
merged_df = merged_df.merge(play_labels, on=['gameId', 'playId'], how='inner')
merged_df.fillna(0, inplace=True)

In [ ]:
merged_df['isConfused'] = merged_df['isConfused'].replace({0: 0, 1: 0.5, 2: 1})

0        0.5
1        0.5
2        0.5
3        0.0
4        0.0
        ... 
12497    0.0
12498    0.0
12499    0.5
12500    0.5
12501    0.0
Name: isConfused, Length: 12502, dtype: float64


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

X = merged_df.drop(columns=['gameId', 'playId', 'isConfused'])
y = merged_df['isConfused']

# ---- Step 2: Train/Test Split ----
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ---- Step 3: Add constant for intercept in statsmodels ----
X_train_sm = sm.add_constant(X_train)  # adds intercept term
X_test_sm = sm.add_constant(X_test)

# ---- Step 4: Fit model ----
model = sm.OLS(y_train, X_train_sm).fit()

# ---- Step 5: Print model summary (p-values, R², coefficients) ----
print(model.summary())

# ---- Step 6: Predict and evaluate ----
y_pred = model.predict(X_test_sm)
mse = mean_squared_error(y_test, y_pred)
r2 = model.rsquared

print(f"\nMSE: {mse:.4f}")
print(f"R² (train): {r2:.4f}")

# ---- Step 7: Feature Importance (absolute value of coefficients) ----
importance = model.params[1:].abs().sort_values(ascending=False)  # skip the constant
print("\nFeature Importance (based on absolute coefficient values):")
print(importance)

                            OLS Regression Results                            
Dep. Variable:             isConfused   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     2.562
Date:                Tue, 25 Mar 2025   Prob (F-statistic):           0.000109
Time:                        10:10:51   Log-Likelihood:                -285.25
No. Observations:               10001   AIC:                             614.5
Df Residuals:                    9979   BIC:                             773.1
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const           

In [41]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

print(merged_df['isConfused'].value_counts())
merged_df = merged_df[merged_df['isConfused'] != 1]
merged_df['isConfused'] = merged_df['isConfused'].replace({2: 1})
# ---- Step 1: Prepare features and target ----
X = merged_df.drop(columns=['gameId', 'playId', 'isConfused', 'Unnamed: 0'])  # drop non-feature columns
y = merged_df['isConfused'] 
print(merged_df['isConfused'].value_counts())

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

logreg = LogisticRegression(solver='lbfgs', max_iter=1000, class_weight='balanced')
logreg.fit(X_train_scaled, y_train)

y_pred = logreg.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

merged_df['isConfusedPredicted'] = logreg.predict(scaler.transform(X))

merged_df.to_csv("~/nfl-data-bowl/Sample_Data/Raw/predicted_confusion_output.csv", index=False)


isConfused
0    5893
1    5822
2     787
Name: count, dtype: int64
isConfused
0    5893
1     787
Name: count, dtype: int64
Accuracy: 0.6145209580838323

Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.64      0.74      1170
           1       0.15      0.45      0.23       166

    accuracy                           0.61      1336
   macro avg       0.52      0.54      0.48      1336
weighted avg       0.80      0.61      0.68      1336


Confusion Matrix:
[[746 424]
 [ 91  75]]


In [42]:
coefficients = logreg.coef_[0]  # [0] because it's binary classification
feature_names = X.columns

# Create DataFrame of feature importance
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefficients,
    'Abs_Coefficient': np.abs(coefficients)
})

# Sort by absolute importance
importance_df = importance_df.sort_values(by='Abs_Coefficient', ascending=False)

print(importance_df)

                         Feature  Coefficient  Abs_Coefficient
8    Moving Defensive After Snap     0.432351         0.432351
12           num_defenders_react    -0.311308         0.311308
16         total_motion_duration     0.252688         0.252688
7    Moving Offensive After Snap    -0.212374         0.212374
5   Moving Offensive Before Snap    -0.180320         0.180320
11             num_motion_events     0.122496         0.122496
2             nearest_defender_3     0.118090         0.118090
6   Moving Defensive Before Snap    -0.102347         0.102347
4             nearest_defender_5    -0.100626         0.100626
17             teamTotalDistance    -0.100383         0.100383
9           segment_switch_count     0.084080         0.084080
3             nearest_defender_4     0.079802         0.079802
15                 var_sim_score     0.060623         0.060623
10                   num_motions     0.042799         0.042799
13                 avg_sim_score     0.020993         0